In [1]:
import talib
import numpy as np
import pandas as pd
import random
from collections import deque

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_small = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df_small.Date = pd.to_datetime(df_small.Date)
df_small.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2020-07-30 07:00:00   1.29543   1.29607  1.29540    1.29572   1.29562   
1 2020-07-30 07:05:00   1.29569   1.29591  1.29505    1.29542   1.29588   
2 2020-07-30 07:10:00   1.29541   1.29590  1.29514    1.29559   1.29557   
3 2020-07-30 07:15:00   1.29561   1.29567  1.29510    1.29515   1.29577   
4 2020-07-30 07:20:00   1.29514   1.29643  1.29501    1.29642   1.29529   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.29623  1.29560    1.29589   1.29552   1.29615  1.29550    1.29580  
1   1.29608  1.29522    1.29560   1.29578   1.29600  1.29514    1.29551  
2   1.29606  1.29535    1.29575   1.29549   1.29598  1.29524    1.29567  
3   1.29584  1.29530    1.29533   1.29569   1.29576  1.29520    1.29524  
4   1.29660  1.29518    1.29660   1.29522   1.29652  1.29510    1.29651  
---------------------------------------------------------------------------
                     Date  Bi

In [5]:
print(df_small.head())
print('-' * 75)
print(df_small.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2020-07-30 07:00:00   1.29543   1.29607  1.29540    1.29572   1.29562   
1 2020-07-30 07:05:00   1.29569   1.29591  1.29505    1.29542   1.29588   
2 2020-07-30 07:10:00   1.29541   1.29590  1.29514    1.29559   1.29557   
3 2020-07-30 07:15:00   1.29561   1.29567  1.29510    1.29515   1.29577   
4 2020-07-30 07:20:00   1.29514   1.29643  1.29501    1.29642   1.29529   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.29623  1.29560    1.29589   1.29552   1.29615  1.29550    1.29580  
1   1.29608  1.29522    1.29560   1.29578   1.29600  1.29514    1.29551  
2   1.29606  1.29535    1.29575   1.29549   1.29598  1.29524    1.29567  
3   1.29584  1.29530    1.29533   1.29569   1.29576  1.29520    1.29524  
4   1.29660  1.29518    1.29660   1.29522   1.29652  1.29510    1.29651  
---------------------------------------------------------------------------
                     Date  Bi

In [7]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema175'] = talib.EMA(df['Mid_Close'], timeperiod=175)
df['ema150'] = talib.EMA(df['Mid_Close'], timeperiod=150)
df['ema125'] = talib.EMA(df['Mid_Close'], timeperiod=125)
df['ema100'] = talib.EMA(df['Mid_Close'], timeperiod=100)
df['ema75'] = talib.EMA(df['Mid_Close'], timeperiod=75)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

KeyError: 'beep_boop'

In [8]:
value_per_pip = 1.0
max_pips_to_risk = 0.0100
amounts_per_day = [-4 / 50000, -5 / 50000, -6 / 50000]

In [9]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_cushion(adx):
    return ((adx - 15) // 5 + 1) / 10000

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, bar_length, spread_cutoff, use_tl, max_pips_to_risk, strict_pips):
    bar_length /= 10000
    max_pips_to_risk /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trade = None

    i = 15

    while i < len(df):
        curr_date = df.loc[df.index[i], 'Date']
        ema200 = df.loc[df.index[i - 1], 'ema200']
        ema175 = df.loc[df.index[i - 1], 'ema175']
        ema150 = df.loc[df.index[i - 1], 'ema150']
        ema125 = df.loc[df.index[i - 1], 'ema125']
        ema100 = df.loc[df.index[i - 1], 'ema100']
        ema75 = df.loc[df.index[i - 1], 'ema75']
        ema50 = df.loc[df.index[i - 1], 'ema50']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        curr_ml2 = df.loc[df.index[i - 2], 'Mid_Low']
        curr_ml1 = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh2 = df.loc[df.index[i - 2], 'Mid_High']
        curr_mh1 = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh1 - curr_ml1)
        beep_boop2 = df.loc[df.index[i - 2], 'macdhist']
        beep_boop1 = df.loc[df.index[i - 1], 'macdhist']
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
          df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                              'Ask_Low', 'Ask_Close']]

        if trade is None:
          if beep_boop2 < 0 and beep_boop1 > 0 and curr_ml2 > ema200 and curr_ml1 > ema200 and curr_ml2 > ema50 and curr_ml1 > ema50 and enough_volatility and curr_bar_length <= bar_length:
              all_buys = True

              if all_buys:
                
                pullback = None
                lowest_low = min(df.loc[df.index[i - 15:i], 'Mid_Low'])
                ema_vals = sorted([ema75, ema100, ema125, ema150, ema175, ema200], reverse=True)

                for ema_val in ema_vals:
                  if ema_val <= lowest_low:
                    pullback = ema_val
                    break

                if pullback is None:
                  # cushion = calculate_cushion(adx)
                  pullback = lowest_low

                if pullback is not None:
                    open_price = float(curr_ask_open)
                    stop_loss = round(pullback, 5)
                    # stop_loss = round(pullback - pullback_cushion, 3)

                    if stop_loss < open_price:
                        if strict_pips:
                          curr_pips_to_risk = open_price - stop_loss

                        else:
                          curr_pips_to_risk = min(open_price - stop_loss, max_pips_to_risk)

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                              'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                              'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_buys += 1

                            pips_risked.append(curr_pips_to_risk)

          elif beep_boop2 == 1 and beep_boop1 == 2 and curr_mh2 < ema200 and curr_mh1 < ema200 and curr_ml2 < ema50 and curr_ml1 < ema50 and enough_volatility and curr_bar_length <= bar_length:
              all_sells = True

              if all_sells:
                pullback = None
                highest_high = max(df.loc[df.index[i - 15:i], 'Mid_High'])
                ema_vals = sorted([ema75, ema100, ema125, ema150, ema175, ema200])

                for ema_val in ema_vals:
                  if ema_val >= highest_high:
                    pullback = ema_val
                    break

                if pullback is None:
                  # cushion = calculate_cushion(adx)
                  pullback = highest_high

                if pullback is not None:
                    open_price = float(curr_bid_open)
                    stop_loss = round(pullback, 5)
                    # stop_loss = round(pullback + pullback_cushion, 3)

                    if stop_loss > open_price:
                        if strict_pips:
                          curr_pips_to_risk = stop_loss - open_price

                        else:
                          curr_pips_to_risk = min(stop_loss - open_price, max_pips_to_risk)

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                    'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_sells += 1

                            pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
          trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          # if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
          #     trade['stop_loss'] = curr_bid_high - trade['pips_risked']
          if trade is not None and trade['original_units'] == trade['n_units'] and trade['trade_type'] == 'buy' and curr_bid_high - trade['open_price'] >= trade['pips_risked']:
              trade['stop_loss'] = trade['open_price']
              reward += trade['pips_risked'] * (trade['n_units'] / 2) * value_per_pip
              day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'] / 2)
              trade['n_units'] = trade['n_units'] / 2
              

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
          trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
          trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          # if trade is not None and trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
          #   trade['stop_loss'] = trade['pips_risked'] + curr_ask_low
          if trade is not None and trade['original_units'] == trade['n_units'] and trade['trade_type'] == 'sell' and trade['open_price'] - curr_ask_low >= trade['pips_risked']:
              trade['stop_loss'] = trade['open_price']
              reward += trade['pips_risked'] * (trade['n_units'] / 2) * value_per_pip
              day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'] / 2)
              trade['n_units'] = trade['n_units'] / 2

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
          trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked

In [10]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1, 1.5, 2]
# risk_reward_ratio_vals = [2]
# bar_lengths = [5, 10, 50]
bar_lengths = [5, 10]
spread_cutoffs = [2, 3]
# spread_cutoffs = [2]
use_tl_vals = [True, False]
pips_to_risk_vals = [15, 20]
strict_pips_vals = [True, False]
n_possibilities = len(risk_reward_ratio_vals) * len(bar_lengths) * len(spread_cutoffs) * len(use_tl_vals) * len(pips_to_risk_vals) * len(strict_pips_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
  for bar_length in bar_lengths:
    for spread_cutoff in spread_cutoffs:
      for use_tl in use_tl_vals:
        for pips_to_risk in pips_to_risk_vals:
          for strict_pips in strict_pips_vals:
            all_combos.append((risk_reward_ratio, bar_length, spread_cutoff, use_tl, pips_to_risk, strict_pips))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_bar_length = None
best_spread_cutoff = None
best_use_tl_val = None
best_pips_to_risk = None
best_strict_pips_val = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, bar_length, spread_cutoff, use_tl, pips_to_risk, strict_pips in combos_to_try:
  reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked = run_simulation(risk_reward_ratio, bar_length, spread_cutoff, use_tl, pips_to_risk, strict_pips)
  runs_finished += 1

  print(reward)
  print('Num buys: ' + str(n_sells))
  print('Num sells: ' + str(n_buys))
  print('Num trades: ' + str(n_buys + n_sells))
  print('Num wins: ' + str(n_wins))
  print('Num losses: ' + str(n_losses))
  print('Win streak: ' + str(win_streak))
  print('Loss streak: ' + str(loss_streak))
  print('Avg pips risked: ' + str(sum(pips_risked) / len(pips_risked)))
  print('Remaining runs: ' + str(n_runs - runs_finished))

  min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
  if min_item is None or reward > min_item['reward']:
    if min_item is not None:
      best_rewards.remove(min_item)
    best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'bar_length': bar_length, 'spread': spread_cutoff, 'use_tl': use_tl, 'pips_to_risk': pips_to_risk, 'strict_pips': strict_pips})


  if reward > best_reward:
    best_reward = reward
    best_risk_reward = risk_reward_ratio
    best_bar_length = bar_length
    best_spread_cutoff = spread_cutoff
    best_use_tl_val = use_tl
    best_pips_to_risk = pips_to_risk
    best_strict_pips_val = strict_pips

  print('Best reward so far: ' + str(best_reward))
  print()

Num runs: 96

-6857.50000000006
Num buys: 0
Num sells: 284
Num trades: 284
Num wins: 92
Num losses: 192
Win streak: 5
Loss streak: 13
Avg pips risked: 0.0013655281690140891
Remaining runs: 95
Best reward so far: -6857.50000000006

-6912.49999999999
Num buys: 0
Num sells: 386
Num trades: 386
Num wins: 161
Num losses: 225
Win streak: 7
Loss streak: 12
Avg pips risked: 0.0016140414507772047
Remaining runs: 94
Best reward so far: -6857.50000000006

-6898.99999999999
Num buys: 0
Num sells: 386
Num trades: 386
Num wins: 161
Num losses: 225
Win streak: 7
Loss streak: 12
Avg pips risked: 0.0016140414507772047
Remaining runs: 93
Best reward so far: -6857.50000000006

-6010.499999999993
Num buys: 0
Num sells: 256
Num trades: 256
Num wins: 56
Num losses: 200
Win streak: 4
Loss streak: 19
Avg pips risked: 0.0013582812500000053
Remaining runs: 92
Best reward so far: -6010.499999999993

-7362.999999999991
Num buys: 0
Num sells: 408
Num trades: 408
Num wins: 182
Num losses: 226
Win streak: 7
Loss str

In [11]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best bar length: ' + str(best_bar_length))
print('Best spread: ' + str(best_spread_cutoff))
print('Best TL val: ' + str(best_use_tl_val))
print('Best pips to risk: ' + str(best_pips_to_risk))
print('Best strict pips val: ' + str(best_strict_pips_val))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)


------------ FINAL RESULTS ------------
Best reward: -2852.500000000062
Best risk/reward ratio: 1.5
Best bar length: 5
Best spread: 2
Best TL val: False
Best pips to risk: 15
Best strict pips val: True
-----------------------
Top results:
{'reward': -3165, 'ratio': 1.5, 'bar_length': 5, 'spread': 3, 'use_tl': True, 'pips_to_risk': 15, 'strict_pips': True}
{'reward': -3278, 'ratio': 2, 'bar_length': 10, 'spread': 3, 'use_tl': False, 'pips_to_risk': 20, 'strict_pips': False}
{'reward': -3158, 'ratio': 1.5, 'bar_length': 5, 'spread': 2, 'use_tl': True, 'pips_to_risk': 15, 'strict_pips': True}
{'reward': -3491, 'ratio': 2, 'bar_length': 10, 'spread': 3, 'use_tl': True, 'pips_to_risk': 15, 'strict_pips': True}
{'reward': -2856, 'ratio': 1.5, 'bar_length': 5, 'spread': 3, 'use_tl': False, 'pips_to_risk': 15, 'strict_pips': True}
{'reward': -3103, 'ratio': 2, 'bar_length': 5, 'spread': 2, 'use_tl': True, 'pips_to_risk': 15, 'strict_pips': True}
{'reward': -3093, 'ratio': 1, 'bar_length': 5, '